# Search for clear Planet Images covering a GeoJSON-specified area, download, and tile into 224x224 squares

## Created by Julian Lee, April 8 2021

In [69]:
## Import Packages - if not installed, use conda or pip to install

import requests
import os
import utm
import rasterio as rio
import shapely.geometry
import datetime


from pathlib import Path
from dateutil.relativedelta import relativedelta
from itertools import product
from rasterio import windows
from rasterio.mask import mask, raster_geometry_mask
from rasterio.io import MemoryFile

In [51]:
## Setup API Authentication

os.environ['PL_API_KEY'] = 'ef1d2935cae84b3cb4e4cadd95a0779e'
PLANET_API_KEY = os.getenv('PL_API_KEY')
url = 'https://api.planet.com/data/v1/'

session = requests.Session()
session.auth = (PLANET_API_KEY, '')

#### User-defined parameters

In [87]:
## Tile width & height
tile_width, tile_height = 224, 224

## Define overlap of tile windows in px
width_overlap, height_overlap = 20, 20

## Relative directory path to save tiles
out_path = './images/Planet/'+ datetime.date.today().isoformat() + '/'

## Number of days in the past to search for clear imagery. Can increase if no results.
days_range = 28

## Minimum image clarity percentage, determined by Planet, to filter. Can lower if no results, but may give cloudy images. 
clear_threshold = 70

## GeoJSON Feature area of interest: replace with your own. GeoJSONs can be generated from drawings at https://www.geojson.io
geojson = {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -63.9242935180664,
              -8.745123882014676
            ],
            [
              -63.92189025878906,
              -8.770572869670541
            ],
            [
              -63.90953063964843,
              -8.768537014766189
            ],
            [
              -63.9151954650879,
              -8.74478455042093
            ],
            [
              -63.91708374023437,
              -8.727308555699958
            ],
            [
              -63.9268684387207,
              -8.729174963125589
            ],
            [
              -63.9242935180664,
              -8.745123882014676
            ]
          ]
        ]
      }
    }

#### Setup search filters

In [63]:
item_types = ['PSScene3Band', 'PSScene4Band', 'PSOrthoTile', 'REOrthoTile', 'REScene', 'SkySatScene', 'SkySatCollect']
end = datetime.datetime.utcnow() # <-- get current day in UTC
start = datetime.datetime.utcnow() - relativedelta(days=days_range)  

fltr = {
    "type": "AndFilter",
    "config": [
        {
            "type": "DateRangeFilter", 
            "field_name":"acquired",
            "config": {
              "gt": start.isoformat("T") + "Z",
              "lte": end.isoformat("T") + "Z"
           }
        },
        {
           "type": "GeometryFilter",
           "field_name":"geometry",
           "config": geojson['features'][0]['geometry']
        },
        {
           "type":"RangeFilter",
           "field_name":"clear_percent",
           "config":{
              "gte": clear_threshold
            }
        },
        {
         "type":"PermissionFilter",
         "config": [
                "assets:download"
            ]
        }
    ]
}

#### Send search to Planet API using given filters and choose the result with best coverage of our AOI

In [81]:
res = session.post(url + 'quick-search', json = {'filter': fltr, 'item_types': item_types})
features = res.json()['features']
aoi = shapely.geometry.shape(geojson['geometry'])

best_feature = None
best_coverage = 0

for feature in features:
    intersection = shapely.geometry.shape(feature['geometry']).intersection(aoi).area 
    if intersection > best_coverage: 
        best_feature = feature
        best_coverage = intersection

#### Send activation request to Planet API for the best result - this prepares the image for download

In [65]:
if best_feature:
    asset = session.get(best_feature['_links']['assets']).json()
    activation_link = asset['analytic']['_links']['activate']
    res = session.post(activation_link)

    ## This should output 200 or 204 
    print('Search successful, activation request status: ', res.status_code)
else:
    print('No search results, widen filters')

Search successful, activation request status:  204


#### Download best image - the activation might take some time to process, so if this doesn't work just wait a few minutes

In [66]:
download_link = session.get(best_feature['_links']['assets']).json()['analytic']['location']
img = session.get(download_link)

#### Convert AOI geometry from lat-long coordinates to UTM coordinates for filtering TIF file

In [85]:
for idx, pair in enumerate(geojson['geometry']['coordinates'][0]):
    x, y, _, _ = utm.from_latlon(pair[1], pair[0])
    geojson['geometry']['coordinates'][0][idx][0] = x
    geojson['geometry']['coordinates'][0][idx][1] = y
geom = shapely.geometry.Polygon(geojson['geometry']['coordinates'][0])

#### Tile the part of the image that covers the AOI into 224x224 TIF files and save 

In [88]:
output_filename = 'tile_{}-{}.tif'
Path(out_path).mkdir(parents=True, exist_ok=True)

def get_tiles(ds, crop, width=224, height=224):
    nols, nrows = crop.width, crop.height
    offsets = product(range(crop.col_off, crop.col_off + nols, width - width_overlap), range(crop.row_off, crop.row_off + nrows, height - height_overlap))
    big_window = windows.Window(col_off=0, row_off=0, width = ds.width, height = ds.height)
    for col_off, row_off in offsets:
        window = windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform
        
with MemoryFile(img.content) as memfile:
    with memfile.open() as inds:
        crop_window = raster_geometry_mask(inds, [geom], crop=True)[2]

        meta = inds.meta.copy()

        for window, transform in get_tiles(inds, crop_window, tile_width, tile_height):
            meta['transform'] = transform
            meta['width'], meta['height'] = window.width, window.height
            outpath = os.path.join(out_path,output_filename.format(int(window.col_off), int(window.row_off)))
            with rio.open(outpath, 'w', **meta) as outds:
                outds.write(inds.read(window=window))